In [ ]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import codecs
import re
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from datetime import date
import firebase_admin
from firebase_admin import credentials, firestore

In [ ]:
driver = webdriver.Chrome(executable_path="/Users/s546761/Documents/chromedriver")
day=date.today().day
x=month=date.today().month
year=date.today().year
s=""

In [ ]:
while month<=x+1:
    while day<=31:
        driver.get("http://calendar.nwmissouri.edu/?d="+str(year)+"-"+str(month)+"-"+str(day))
        sleep(2)
        b=driver.find_elements(By.CLASS_NAME,"event-data")
        c=driver.find_elements(By.CLASS_NAME,"event-listing-name")#.find_elements(By.TAG_NAME,"a")
        l=[]
        for y in range(len(c)):
            c=driver.find_elements(By.CLASS_NAME,"event-listing-name")
            c[y].find_element(By.TAG_NAME,"a").click() 
            sleep(1)
            a=driver.find_elements(By.CLASS_NAME,"content")
            for w in a:
                s+=w.text+"<br>"
            driver.back()
            sleep(1)
        day+=3
    month+=1
    day=1

In [ ]:
d={"name":[],"date":[],"description":[]}
a="";c=0
for i in range(len(l)-1):
    i=c
    if i>=len(l):
        break
    a=l[i]
    for j in range(i+1,len(l)-1):
        if "2023" not in l[j]:
            a+=l[j]
        else:
            d["name"].append(a)
            i=j
                break
    a=""
    if "2023" in l[i]:
        a+=l[i]
    i+=1
    if "2023" in l[i]:
        a+=l[i]
    d["date"].append(a)
    a=""
    i+=1
    for j in range(i,len(l)):
        if "Calendars:"  not in l[j]:
            a+=l[j]+"\n"
        else: 
            a+=l[j]
            i=j
            break
    d["description"].append(a)
    c=i+1
     
c=0
for i in range(len(d["name"])-1):
    for j in range(i+1,len(d["name"])):
        if i<len(d["name"]) and j<len(d["name"])-1:
            if d["name"][i]==d["name"][j] and d["date"][i]==d["date"][j] and d["description"][i]==d["description"][j]:
                d["name"].remove(d["name"][j])
                d["date"].remove(d["date"][j])
                d["description"].remove(d["description"][j])
                    
cred = credentials.Certificate("C:\serviceAccount.json")
firebase_admin.initialize_app(cred)
  
db = firestore.client()
  
data = db.collection(u'data').document(u'events')
data.update({u'name': d["name"], u'date':d["date"], u'description':d["description"]})